In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
path = '/content/drive/MyDrive/ENSIIE/M1/S4/PRR-Projet-Recherche/projet_recherche'
os.chdir(path)

In [3]:
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("clean/param.csv")

In [5]:
df.head()

,Unnamed: 0,m,T,r,rho,kappa,gamma,nu,nu0,price
0,0,0.766060,0.906209,0.004900,-0.267169,0.168065,0.285491,0.418131,0.055328,2.303577
1,3,0.949645,0.243284,0.041703,-0.688789,1.551546,0.791578,0.356502,0.432486,10.468787
2,4,0.678250,0.177665,0.011544,-0.879075,2.699666,0.171349,0.087476,0.285393,0.218416
3,11,1.010499,0.340049,0.046198,-0.161449,1.371800,0.113999,0.053605,0.303124,12.835901
4,15,0.745336,0.201213,0.005276,-0.116926,2.183879,0.347566,0.167538,0.207366,0.710479


In [6]:
df.columns

Index(['Unnamed: 0', 'm', 'T', 'r', 'rho', 'kappa', 'gamma', 'nu', 'nu0',
       'price'],
      dtype='object')

In [7]:
X = df.drop([ 'Unnamed: 0', 'price'], 1)
# X_stand =(X-X.mean())/X.std()
y = df['price']
# y_norm = (y-y.mean())/y.std()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [8]:
X.shape

(37, 8)

In [9]:
# The dataset should be split into 10-validation, 10-testing, 80-training 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

# train is now 80% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=1)

# test is now 10% of the initial data set
# validation is now 10% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=1)

In [11]:
# epochs is the number of iterations over the data
# verbose is a parameter to obtain additional information 
# Note: define the batch size to 
initializer = tf.keras.initializers.GlorotUniform()
model = tf.keras.Sequential([
    keras.layers.Dense(units=200,input_shape=(8,),  kernel_initializer=initializer, activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=200,activation='relu'),
    keras.layers.Dense(units=1),
    ])
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1800      
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 200)               40200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 1)                 201       
                                                                 
Total params: 122,601
Trainable params: 122,601
Non-trainable params: 0
_________________________________________________________________


In [12]:
# The epochs shouls be 8000, but we'll start with something lighter to test
history= model.fit(X_train, y_train, epochs=8000, batch_size= 1024, verbose=1)

Se han truncado las últimas 5000 líneas del flujo de salida.
1/1 [==============================] - 0s 15ms/step - loss: 8.7666e-13
Epoch 5502/8000
1/1 [==============================] - 0s 14ms/step - loss: 2.1782e-12
Epoch 5503/8000
1/1 [==============================] - 0s 22ms/step - loss: 2.0018e-12
Epoch 5504/8000
1/1 [==============================] - 0s 17ms/step - loss: 1.0862e-11
Epoch 5505/8000
1/1 [==============================] - 0s 20ms/step - loss: 2.1964e-11
Epoch 5506/8000
1/1 [==============================] - 0s 18ms/step - loss: 8.5352e-11
Epoch 5507/8000
1/1 [==============================] - 0s 14ms/step - loss: 1.3497e-09
Epoch 5508/8000
1/1 [==============================] - 0s 17ms/step - loss: 2.6784e-08
Epoch 5509/8000
1/1 [==============================] - 0s 13ms/step - loss: 5.3256e-07
Epoch 5510/8000
1/1 [==============================] - 0s 15ms/step - loss: 1.0654e-05
Epoch 5511/8000
1/1 [==============================] - 0s 16ms/step - loss: 2.1353e-0

In [13]:
# y_pred = model.predict(x_test)
y_pred = model.predict(X_test)

In [14]:
y_test

35    24.868769
3     12.835901
29     4.811722
21    33.754849
Name: price, dtype: float64

In [15]:
y_pred

array([[10.849643 ],
       [ 5.1057734],
       [12.496215 ],
       [15.526011 ],
       [16.220736 ],
       [ 9.097626 ],
       [30.731478 ],
       [23.536852 ]], dtype=float32)

In [16]:
from functions import MAPE, MSE, MAE

In [17]:
MAPE(y_test, y_pred)

7.32446

In [18]:
MSE(y_test, y_pred)

1564.59678

In [19]:
MAE(y_test, y_pred)

91.78052